# Loss Functions

In this exercise, you will compare the effects of Loss functions on a `LinearRegression` model.

👇 Import the data from the attached csv file

In [1]:
import pandas as pd

data = pd.read_csv("data.csv")

data.sample(5)

,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Glazing Area,Average Temperature
20,0.76,661.5,416.5,122.50,7.0,0.0,27.280
755,0.69,735.0,294.0,220.50,3.5,0.4,15.475
626,0.98,514.5,294.0,110.25,7.0,0.4,33.045
41,0.64,784.0,343.0,220.50,3.5,0.0,13.670
543,0.82,612.5,318.5,147.00,7.0,0.4,31.095


🎯 Your task is to predict the average temperature inside a greenhouse based on its design. Your temperature predictions will help you select the appropriate greenhouse design for each plant, based on their climate needs. 

🌿 You know that plants can handle small temperature variations, but are exponentially more sensitive as the temperature variations increase. 

## 1. Theory 

❓ Theoretically, which Loss function would you train your model on to limit the risk of killing plants?

<details>
<summary> 🆘 Answer </summary>
    
By theory, you would use a Mean Square Error (MSE) Loss function. It would penalize outlier predictions and prevent your model from committing large errors. This would ensure smaller temperature variations and a lower risk for plants.

</details>

## 2. Application

### 2.1 Preprocessing

👇 Scale the features

In [2]:
from sklearn.preprocessing import StandardScaler

# Select only the features 
X = data.loc[:,'Relative Compactness':'Glazing Area']

# Fit scaler
scaler = StandardScaler().fit(X)

# Scale continuous features 
X_scaled = scaler.transform(X)

### 2.2 Modeling

In this section, you are going to verify the theory by evaluating models optimized on different Loss functions.

### Least Squares (MSE) Loss

👇 **10-Fold Cross-validate** a Linear Regression model optimized by **Stochastic Gradient Descent** (SGD) on a **Least Squares Loss** (MSE)



In [3]:
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.linear_model import SGDRegressor

# Squared loss SGD Regressor
sgd_model = SGDRegressor(loss="squared_error")

# Cross Validate Model
sgd_model_cv = cross_validate(sgd_model, 
                              X_scaled, 
                              data['Average Temperature'],
                              cv = 10, 
                              scoring = ['r2','max_error'] )
sgd_model_cv

{'fit_time': array([0.00857854, 0.00512004, 0.00587296, 0.00732732, 0.00757599,
        0.00478077, 0.00512648, 0.00785351, 0.00583291, 0.00460458]),
 'score_time': array([0.00145721, 0.00101519, 0.0006721 , 0.00075698, 0.00114679,
        0.00113797, 0.00100422, 0.00084329, 0.00074148, 0.00069904]),
 'test_r2': array([0.78640565, 0.90881163, 0.89578972, 0.88404789, 0.93127231,
        0.89661013, 0.92725322, 0.91579246, 0.89478684, 0.93968275]),
 'test_max_error': array([-9.86848357, -8.69739583, -8.7680036 , -9.18390341, -8.89623249,
        -8.65797407, -8.50389647, -8.80053997, -8.43604891, -7.62026584])}

👇 Compute 
- the mean cross-validated R2 score `r2`
- the single biggest prediction error in °C of all your folds `max_error`?

(Tips: `max_error` is an accepted scoring metrics in sklearn)

In [4]:
r2 = sgd_model_cv['test_r2'].mean()
r2

0.8980452601598143

In [5]:
max_error = abs(sgd_model_cv['test_max_error']).max()
max_error

9.868483570617595

### Mean Absolute Error (MAE) Loss

What if we optimize our model on the MAE instead?

👇 **10-Fold Cross-validate** a Linear Regression model optimized by **Stochastic Gradient Descent** (SGD) on a **MAE** Loss

<details>
<summary>💡 Hints</summary>

- MAE loss cannot be directly specified in `SGDRegressor`. It must be engineered by adjusting the right parameters

</details>

In [6]:
# MAE loss engineered by setting epsilon_insensitive = 0
mae_model = SGDRegressor(loss="epsilon_insensitive", epsilon = 0)

# Cross Validate Model
mae_sgd = cross_validate(mae_model, 
                         X_scaled, 
                         data['Average Temperature'], 
                         cv = 10,  
                         scoring = ['r2','max_error'])

👇 Compute 
- the mean cross-validated R2 score `r2_mae`
- the single biggest prediction error of all your folds `max_error_mae`?

In [7]:
r2_mae = mae_sgd['test_r2'].mean()
r2_mae

0.8763559250578533

In [8]:
max_error_mae = abs(mae_sgd['test_max_error']).max()
max_error_mae

11.168003869679481

## 3. Conclusion

❓Which of the models you evaluated seems the most appropriate for your task?

<details>
<summary> 🆘Answer </summary>
    
Although mean cross-validated r2 scores are approximately similar between the two models, the one optimized on a MAE has more chance to make larger mistakes from time to time, increasing the risk of killing plants!

    
</details>

# 🏁 Check your code

In [9]:
from nbresult import ChallengeResult

result = ChallengeResult('loss_functions',
    r2 = r2,
    r2_mae = r2_mae,
    max_error = max_error,
    max_error_mae = max_error_mae,                     
)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.8.10, pytest-7.1.2, pluggy-1.0.0 -- /bin/python3
cachedir: .pytest_cache
rootdir: /home/quantium/labs/lewagon/data-challenges/05-ML/04-Under-the-hood/01-Loss-Functions/tests
plugins: dash-1.19.0, anyio-3.6.1
collecting ... collected 3 items

test_loss_functions.py::TestLossFunctions::test_max_error_order PASSED   [ 33%]
test_loss_functions.py::TestLossFunctions::test_r2 PASSED                [ 66%]
test_loss_functions.py::TestLossFunctions::test_r2_mae PASSED            [100%]

============================== 3 passed in 0.15s ===============================


💯 You can commit your code:

git add tests/loss_functions.pickle

git commit -m 'Completed loss_functions step'

git push origin master

